In [1]:
# Familiar imports
import numpy as np
import pandas as pd
import random
import os
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

import lightgbm as lgb
#import xgboost as xgb
#import catboost as ctb

import warnings
warnings.simplefilter('ignore')

# Parameters

In [2]:
target = 'target'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 17
    CVSEED = 17
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 10
    N_ESTIMATORS = 20000
    EARLY_STOPPING_ROUNDS = 200
    VERBOSE = 1000
    SEED = 17
    CVSEED = 17
    #N_ITERS = 10

In [3]:
def set_seed(seed=17):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

# Load data

In [4]:
# Load the training data
train = pd.read_csv("../input/30-days-of-ml/train.csv")
test = pd.read_csv("../input/30-days-of-ml/test.csv")

# Preview the data
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,C,B,B,A,E,C,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,A,A,B,D,A,F,A,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,A,C,B,D,A,D,A,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,A,C,B,D,A,E,C,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,A,C,B,D,A,E,A,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [5]:
# Low MI scores
for pp in ["cat2","cat3","cat4","cat6"]:
    train.pop(pp)
    test.pop(pp)
train.head()

,id,cat0,cat1,cat5,cat7,cat8,cat9,cont0,cont1,cont2,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,E,C,N,0.201470,-0.014822,0.669699,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,D,F,A,O,0.743068,0.367411,1.021605,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,D,D,A,F,0.742708,0.310383,-0.012673,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,D,E,C,K,0.429551,0.620998,0.577942,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,D,E,A,N,1.058291,0.367492,-0.052389,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [6]:
# Separate target from features
y = train['target']
features = train.drop(['id','target'], axis=1)

# Preview features
features.head()

,cat0,cat1,cat5,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,B,B,B,E,C,N,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
1,B,B,D,F,A,O,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
2,A,A,D,D,A,F,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
3,B,B,D,E,C,K,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
4,A,A,D,E,A,N,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


# Preprocessing

In [7]:
'''
# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# ordinal-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()
ordinal_encoder = OrdinalEncoder()
X[object_cols] = ordinal_encoder.fit_transform(features[object_cols])
X_test[object_cols] = ordinal_encoder.transform(test[object_cols])

# Preview the ordinal-encoded features
X.head()
'''

"\n# List of categorical columns\nobject_cols = [col for col in features.columns if 'cat' in col]\n\n# ordinal-encode categorical columns\nX = features.copy()\nX_test = test.drop(['id'], axis=1).copy()\nordinal_encoder = OrdinalEncoder()\nX[object_cols] = ordinal_encoder.fit_transform(features[object_cols])\nX_test[object_cols] = ordinal_encoder.transform(test[object_cols])\n\n# Preview the ordinal-encoded features\nX.head()\n"

In [8]:

# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# one-hot-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()

oh_encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
X_ohe = oh_encoder.fit_transform(features[object_cols])
X_test_ohe = oh_encoder.transform(test[object_cols])

X_ohe = pd.DataFrame(X_ohe, columns=[f"ohe_{i}" for i in range(X_ohe.shape[1])])
X_test_ohe = pd.DataFrame(X_test_ohe, columns=[f"ohe_{i}" for i in range(X_test_ohe.shape[1])])

X = pd.concat([X, X_ohe], axis=1)
X_test = pd.concat([X_test, X_test_ohe], axis=1)
X = X.drop(object_cols, axis=1)
X_test = X_test.drop(object_cols, axis=1)
    
# Preview the one-hot-encoded features
X.head()


,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,...,ohe_28,ohe_29,ohe_30,ohe_31,ohe_32,ohe_33,ohe_34,ohe_35,ohe_36,ohe_37
0,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
scale_features = [col for col in features.columns if 'cont' in col]

ss = StandardScaler()
X[scale_features] = ss.fit_transform(features[scale_features])
X_test[scale_features] = ss.transform(test[scale_features])

In [10]:
# Swap noise

# Random
def apply_noise_rn(df, p=.75):
    should_not_swap = np.random.binomial(1, p, df.shape)
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Row-wise
def apply_noise_row(df, p=.75):
    should_not_swap = np.zeros(df.shape)
    for i in range(df.shape[0]):
        for j in np.random.choice(df.shape[1],int(p*df.shape[1]),replace=False):
            should_not_swap[i,j]=1 
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Optuna

In [11]:
# Optuna for parameter search
!pip install -q optuna

import optuna
import pickle

In [12]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial, X=X, y=y):
  """
  """
  param_space = {
               #'device':'gpu',  # Use GPU acceleration
               #'boosting_type': 'gbdt',
               'reg_lambda':trial.suggest_uniform('reg_lambda', 58.8, 61.8),
              'reg_alpha':trial.suggest_uniform('reg_alpha', 44, 47),
                'subsample': trial.suggest_uniform('subsample', 0.371, 0.421),
              'colsample_bytree':trial.suggest_uniform('colsample_bytree', 0.0202, 0.0602),
                #'subsample_freq': trial.suggest_int('subsample_freq', 1, 10),
               'learning_rate':trial.suggest_uniform('learning_rate', 9.69e-2, 1.169e-1),
                 'min_child_samples':trial.suggest_int('min_child_samples', 15, 20),
              'num_leaves':trial.suggest_int('num_leaves', 206, 218),
              'max_depth':trial.suggest_int('max_depth', 23, 25),
              #'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 0.005),
              #'class_weight':trial.suggest_categorical('class_weight',['balanced',None]),
               'n_estimators':N_ESTIMATORS,
                'n_jobs' : -1,
              'metric':'rmse',
              'max_bin':trial.suggest_int('max_bin', 1003, 1053),
              'cat_smooth':trial.suggest_int('cat_smooth', 82, 87),
              'cat_l2':trial.suggest_loguniform('cat_l2', 23.4, 29.4)
                }
            
  seed_list=[SEED, SEED+1]
  #kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
  #lgb_oof = np.zeros(train.shape[0])
  #lgb_pred = np.zeros(test.shape[0])
  X_tr, X_va, y_tr, y_va = train_test_split(X,y,test_size=.2,random_state=CVSEED)
  pred=np.zeros(y_va.shape[0])
    
  #for (trn_idx, val_idx) in enumerate(kf.split(X, y)):
  #print(f"===== fold {fold} =====")
    
  #X_tr = apply_noise_row(X_tr)

  
  #start = time.time()
  for inseed in seed_list:
    param_space['random_state'] = inseed

    model = lgb.LGBMRegressor(**param_space)
    model.fit(
        X_tr, 
        y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric='rmse',
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        verbose=VERBOSE,
        #categorical_feature=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
        #callbacks=[optuna.integration.LightGBMPruningCallback(trial, metric='rmse')],
    )


    pred += model.predict(X_va) / len(seed_list)
    
  #elapsed = time.time() - start
  rmse = mean_squared_error(y_va, pred, squared=False)
  #print(f"fold {fold} - lgb rmse: {rmse:.6f}, elapsed time: {elapsed:.2f}sec\n")            
  
  #print(f"oof lgb_rmse = {mean_squared_error(train[target], lgb_oof, squared=False)}")

  
  return rmse

In [13]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-08-29 15:49:52,605] A new study created in memory with name: no-name-df479054-1155-4f31-a7d0-24fdb5b8a760


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.71528
[2000]	valid_0's rmse: 0.71462
Early stopping, best iteration is:
[2158]	valid_0's rmse: 0.714573
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.714556
[2000]	valid_0's rmse: 0.714163
Early stopping, best iteration is:
[1974]	valid_0's rmse: 0.714153


[I 2021-08-29 15:50:48,254] Trial 0 finished with value: 0.7140660417309733 and parameters: {'reg_lambda': 59.37247800746857, 'reg_alpha': 46.74740903144017, 'subsample': 0.40314841433425597, 'colsample_bytree': 0.047674922288663046, 'learning_rate': 0.11295311366478794, 'min_child_samples': 20, 'num_leaves': 209, 'max_depth': 24, 'max_bin': 1019, 'cat_smooth': 82, 'cat_l2': 27.53807872810818}. Best is trial 0 with value: 0.7140660417309733.


Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[1]	valid_0's rmse: 8.27805
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.715723
[2000]	valid_0's rmse: 0.714607
[3000]	valid_0's rmse: 0.714425
[4000]	valid_0's rmse: 0.714348
[5000]	valid_0's rmse: 0.714321
[6000]	valid_0's rmse: 0.714308
[7000]	valid_0's rmse: 0.714302
[8000]	valid_0's rmse: 0.714299
[9000]	valid_0's rmse: 0.714299
[10000]	valid_0's rmse: 0.714298
[11000]	valid_0's rmse: 0.714298
[12000]	valid_0's rmse: 0.714297
[13000]	valid_0's rmse: 0.714297
[14000]	valid_0's rmse: 0.714297
[15000]	valid_0's rmse: 0.714297
[16000]	valid_0's rmse: 0.714297
[17000]	valid_0's rmse: 0.714297
[18000]	valid_0's rmse: 0.714297
[19000]	valid_0's rmse: 0.714297
[20000]	valid_0's rmse: 0.714297
Did not meet early stopping. Best iteration is:
[19990]	valid_0's rmse: 0.714297


[I 2021-08-29 15:52:16,007] Trial 1 finished with value: 0.7215775071972174 and parameters: {'reg_lambda': 61.11235114992489, 'reg_alpha': 46.58635961894863, 'subsample': 0.388947023473248, 'colsample_bytree': 0.022255664371316056, 'learning_rate': 0.10797340596899264, 'min_child_samples': 16, 'num_leaves': 208, 'max_depth': 23, 'max_bin': 1011, 'cat_smooth': 86, 'cat_l2': 29.155145134113567}. Best is trial 0 with value: 0.7140660417309733.


...

Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.715245
[2000]	valid_0's rmse: 0.714577
Early stopping, best iteration is:
[2209]	valid_0's rmse: 0.714533
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.714444
[2000]	valid_0's rmse: 0.714073
Early stopping, best iteration is:
[2017]	valid_0's rmse: 0.714071


[I 2021-08-29 16:16:05,464] Trial 28 finished with value: 0.7139903895722409 and parameters: {'reg_lambda': 59.535735282464216, 'reg_alpha': 44.17262044748552, 'subsample': 0.39326014116962515, 'colsample_bytree': 0.03850972760592158, 'learning_rate': 0.11096731683506365, 'min_child_samples': 17, 'num_leaves': 209, 'max_depth': 24, 'max_bin': 1049, 'cat_smooth': 87, 'cat_l2': 28.43849507330949}. Best is trial 28 with value: 0.7139903895722409.


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.715219
[2000]	valid_0's rmse: 0.714584
Early stopping, best iteration is:
[2196]	valid_0's rmse: 0.714553
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.714486
[2000]	valid_0's rmse: 0.714075
Early stopping, best iteration is:
[1974]	valid_0's rmse: 0.714068


[I 2021-08-29 16:17:00,464] Trial 29 finished with value: 0.7139779682129103 and parameters: {'reg_lambda': 59.4077902110251, 'reg_alpha': 44.16098510640907, 'subsample': 0.403195864806355, 'colsample_bytree': 0.039533970364454794, 'learning_rate': 0.11384998791349592, 'min_child_samples': 16, 'num_leaves': 213, 'max_depth': 24, 'max_bin': 1026, 'cat_smooth': 87, 'cat_l2': 29.360702296801012}. Best is trial 29 with value: 0.7139779682129103.


Number of finished trials: 30
Best trial: {'reg_lambda': 59.4077902110251, 'reg_alpha': 44.16098510640907, 'subsample': 0.403195864806355, 'colsample_bytree': 0.039533970364454794, 'learning_rate': 0.11384998791349592, 'min_child_samples': 16, 'num_leaves': 213, 'max_depth': 24, 'max_bin': 1026, 'cat_smooth': 87, 'cat_l2': 29.360702296801012}


In [14]:
study.best_params

{'reg_lambda': 59.4077902110251,
 'reg_alpha': 44.16098510640907,
 'subsample': 0.403195864806355,
 'colsample_bytree': 0.039533970364454794,
 'learning_rate': 0.11384998791349592,
 'min_child_samples': 16,
 'num_leaves': 213,
 'max_depth': 24,
 'max_bin': 1026,
 'cat_smooth': 87,
 'cat_l2': 29.360702296801012}

# Log

====== Ordinal encoding =========

0.7169747685796982 no noise ver1

0.716813599897907 no noise ver3 (narrow space) final



====== One-hot encoding =========

0.724412184059166 no noise ver2

0.7148014527408726 no noise ver6

0.7141152381854105 no noise ver7 (narrow space)

0.7140553164806666 no noise ver8 (narrow space)

0.7140062352122274 no noise ver9 (narrow space)